# Recon2.2plusthreeCompetition).json [Competitive mechanism only]

##Changing medium

In [1]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba

In [2]:
M = cobra.io.load_json_model('Recon2.2plusthreeCompetitive.json')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [3]:
# deleting brain medium composition because this metabolite will uptake from blood
M.reactions.EX_co2_e_INB.lower_bound=0
M.reactions.EX_co2_e.lower_bound=0
M.reactions.EX_glc_e_INB.lower_bound=0
M.reactions.EX_h_e.lower_bound=0
M.reactions.EX_h_e_INB.lower_bound=0
M.reactions.EX_hco3_e_INB.lower_bound=0
M.reactions.EX_hco3_e.lower_bound=0
M.reactions.EX_his_L_e_INB.lower_bound=0
M.reactions.EX_ile_L_e_INB.lower_bound=0
M.reactions.EX_leu_L_e_INB.lower_bound=0
M.reactions.EX_lys_L_e_INB.lower_bound=0
M.reactions.EX_met_L_e_INB.lower_bound=0
M.reactions.EX_nh4_e_INB.lower_bound=0
M.reactions.EX_o2_e_INB.lower_bound=0
M.reactions.EX_phe_L_e_INB.lower_bound=0
M.reactions.EX_pi_e_INB.lower_bound=0
M.reactions.EX_so4_e_INB.lower_bound=0
M.reactions.EX_thr_L_e_INB.lower_bound=0
M.reactions.EX_trp_L_e_INB.lower_bound=0
M.reactions.EX_val_L_e_INB.lower_bound=0

In [4]:
M.solver='glpk'
sol = M.optimize()
sol.objective_value

0.16888753778858653

## Tyrosine is not enough in the diet (model1)

In [15]:
import cobra
import pandas as pd
import numpy as np

In [16]:
from cobra.flux_analysis import pfba
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon2.2plusthreeCompetitivemap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

model=M.copy()
model1=model.copy()

In [17]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = model1.problem.Objective(model1.reactions.biomass_reaction_INB.flux_expression+
                                              model1.reactions.get_by_id('3HLYTCL_INB').flux_expression+
                                              model1.reactions.get_by_id('5HLTDL_INB').flux_expression,
    direction='max')
model1.objective = quadratic_objective

In [18]:
# Changing the phe and tyr in the medium, because they also use for producing dopa.
model1.reactions.EX_phe_L_e.lower_bound=-0.1
model1.reactions.EX_phe_L_e.upper_bound=-0.1
model1.reactions.EX_tyr_L_e.lower_bound=-0.04

In [19]:
model1.reactions.EX_dopa_e_INB.upper_bound=1000
model1.reactions.EX_adrnl_e_INB.upper_bound=1000
model1.reactions.EX_srtn_e_INB.upper_bound=1000
## Other reactions to produce l-dopa is forbidded
model1.reactions.TYRDOPO_INB.upper_bound=0
model1.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model1.reactions.biomass_reaction.lower_bound=0.05
model1.reactions.biomass_reaction.upper_bound=0.05
## do not need too much biomass in the brain
model1.reactions.biomass_reaction_INB.upper_bound=0.1
# No phenylpyruvate exchange in the brain
model1.reactions.get_by_id('EX_phpyr_e_INB').upper_bound=0
# Serotonin production
model1.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
### No PAH in the brain
model1.reactions.PHETHPTOX2_INB.lower_bound=0
model1.reactions.PHETHPTOX2_INB.upper_bound=0
model1.reactions.r0399_INB.lower_bound=0
model1.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
model1.reactions.EX_TH_B_INB.upper_bound=1000
model1.reactions.LdopaTR_TOBD.upper_bound=0 ##No levodopa from liver to blood.
model1.reactions.Ti_To.upper_bound=0.198
model1.solver='glpk'
sol = model1.optimize()
sol.objective_value
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

0.2547554000000002

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [20]:
model2=model1.copy()
# Kncoking gene 'HGNC:8582' to simulate the PKU patients
model2.genes.get_by_id('HGNC:8582').knock_out()
model2.reactions.PHPYRte.upper_bound=0
model2.reactions.PHPYRte.lower_bound=0
sol = model2.optimize()
sol.objective_value
fbasol = model2.optimize()
b = show_map(fbasol,map_loc)
b

0.1873451808343353

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [21]:
## Only adding tyrosine
model3=model2.copy()
model3.reactions.EX_tyr_L_e.lower_bound=-0.1
sol = model3.optimize()
sol.objective_value
fbasol = model3.optimize()
b = show_map(fbasol,map_loc)
b

0.18734518083433524

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [22]:
## Phe deprivation and Tyr addition
model3.reactions.EX_phe_L_e.upper_bound=-0.04
model3.reactions.EX_phe_L_e.lower_bound=-0.04
sol = model3.optimize()
sol.objective_value
fbasol = model3.optimize()
b = show_map(fbasol,map_loc)
b

0.25367589999999995

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [23]:
#Only removing the Phe
model4=model2.copy()
model4.reactions.EX_phe_L_e.upper_bound=-0.04
model4.reactions.EX_phe_L_e.lower_bound=-0.04
sol = model4.optimize()
sol.objective_value
fbasol = model4.optimize()
b = show_map(fbasol,map_loc)
b

0.2140535999999998

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [24]:
model5=model2.copy()
# Taking more Phe for PKU patient to show protein synthesis in the brain
model5.reactions.EX_phe_L_e.lower_bound=-0.11
model5.reactions.EX_phe_L_e.upper_bound=-0.11
sol = model5.optimize()
sol.objective_value
fbasol = model5.optimize()
b = show_map(fbasol,map_loc)
b

0.12953369253538077

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

### Dopamine is more important than serotonin.

##Only competitive mechanism could address most symptoms of PKU disease and treatments. But if dopamine is more important than serotonin in the brain which I will show more objective proportion, only competitive mechanism still could not solve the improvement from only adding tyrosine. Because the system will sacrifice the production of serotonin. Now we should combine the toxicity mechanism. This is why we combine these two mechanism to address PKU disease.

In [25]:
from cobra.flux_analysis import pfba
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon2.2plusthreeCompetitivemap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

model=M.copy()
model1=model.copy()

In [26]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = model1.problem.Objective(model1.reactions.biomass_reaction_INB.flux_expression+
                                              2*model1.reactions.get_by_id('3HLYTCL_INB').flux_expression+
                                              model1.reactions.get_by_id('5HLTDL_INB').flux_expression,
    direction='max')
model1.objective = quadratic_objective

In [27]:
# Changing the phe and tyr in the medium, because they also use for producing dopa.
model1.reactions.EX_phe_L_e.lower_bound=-0.1
model1.reactions.EX_phe_L_e.upper_bound=-0.1
model1.reactions.EX_tyr_L_e.lower_bound=-0.04

In [28]:
model1.reactions.EX_dopa_e_INB.upper_bound=1000
model1.reactions.EX_adrnl_e_INB.upper_bound=1000
model1.reactions.EX_srtn_e_INB.upper_bound=1000
## Other reactions to produce l-dopa is forbidded
model1.reactions.TYRDOPO_INB.upper_bound=0
model1.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model1.reactions.biomass_reaction.lower_bound=0.05
model1.reactions.biomass_reaction.upper_bound=0.05
## do not need too much biomass in the brain
model1.reactions.biomass_reaction_INB.upper_bound=0.1
# minimal serotonin production
# model1.reactions.get_by_id('5HLTDL_INB').lower_bound=0.06
### No PAH in the brain
model1.reactions.PHETHPTOX2_INB.lower_bound=0
model1.reactions.PHETHPTOX2_INB.upper_bound=0
model1.reactions.r0399_INB.lower_bound=0
model1.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
model1.reactions.EX_TH_B_INB.upper_bound=1000
model1.reactions.LdopaTR_TOBD.upper_bound=0 ##No levodopa from liver to blood.
model1.reactions.Ti_To.upper_bound=0.198
model1.reactions.TH_AToB_INB.upper_bound=0.058
model1.solver='glpk'
sol = model1.optimize()
sol.objective_value
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

0.3127554000000001

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [29]:
model2=model1.copy()
# Kncoking gene 'HGNC:8582' to simulate the PKU patients
model2.genes.get_by_id('HGNC:8582').knock_out()
model2.reactions.PHPYRte.upper_bound=0
model2.reactions.PHPYRte.lower_bound=0
sol = model2.optimize()
fluxP=sol.fluxes.loc['PheTR_TOB']
print(fluxP)
if fluxP>=0.08:
    model2.reactions.Phe_TH_INB.lower_bound=0.055
sol = model2.optimize()
sol.objective_value
fbasol = model2.optimize()
b = show_map(fbasol,map_loc)
b

0.08702649999999976


0.19667590000000001

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [30]:
model3=model2.copy()
model3.reactions.EX_tyr_L_e.lower_bound=-0.1
sol = model3.optimize()
sol.objective_value
fba = model3.optimize()
b = show_map(fba,map_loc)
b

0.19667590000000001

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [31]:
model4=model1.copy()
# Kncoking gene 'HGNC:8582' to simulate the PKU patients
model4.genes.get_by_id('HGNC:8582').knock_out()
model4.reactions.EX_phpyr_e_INB.upper_bound=0
model4.reactions.PHPYRte.upper_bound=0
model4.reactions.PHPYRte.lower_bound=0
sol = model2.optimize()
sol.objective_value
fbasol = model2.optimize()
b = show_map(fbasol,map_loc)
b

0.19667590000000001

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [32]:
model5=model4.copy()

In [33]:
model5.reactions.EX_tyr_L_e.lower_bound=-0.1
fba = model5.optimize()
fba.objective_value
b = show_map(fba,map_loc)
b

0.25167590000000006

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

## Tyrosine is enough in the diet

In [80]:
model = M.copy()
from cobra.flux_analysis import pfba
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon2.2plusthreeCCompetitivemap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

model=M.copy()

In [81]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = model.problem.Objective(model.reactions.biomass_reaction_INB.flux_expression+
                                              model.reactions.get_by_id('3HLYTCL_INB').flux_expression+
                                              model.reactions.get_by_id('5HLTDL_INB').flux_expression,
    direction='max')
model.objective = quadratic_objective

In [82]:
# Changing the phe and tyr in the medium, because they also use for producing dopa.
model.reactions.EX_phe_L_e.lower_bound=-0.1
model.reactions.EX_phe_L_e.upper_bound=-0.1
model.reactions.EX_tyr_L_e.lower_bound=-0.1

In [83]:
model.reactions.EX_dopa_e_INB.upper_bound=1000
model.reactions.EX_adrnl_e_INB.upper_bound=1000
model.reactions.EX_srtn_e_INB.upper_bound=1000
## Other reactions to produce l-dopa is forbidded
model.reactions.TYRDOPO_INB.upper_bound=0
model.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model.reactions.biomass_reaction.lower_bound=0.05
model.reactions.biomass_reaction.upper_bound=0.05
## do not need too much biomass in the brain
model.reactions.biomass_reaction_INB.upper_bound=0.1
# No phenylpyruvate exchange in the brain
model.reactions.get_by_id('EX_phpyr_e_INB').upper_bound=0
# Serotonin production
model.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
### No PAH in the brain
model.reactions.PHETHPTOX2_INB.lower_bound=0
model.reactions.PHETHPTOX2_INB.upper_bound=0
model.reactions.r0399_INB.lower_bound=0
model.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
model.reactions.EX_TH_B_INB.upper_bound=1000
model.reactions.LdopaTR_TOBD.upper_bound=0 ##No levodopa from liver to blood.
model.reactions.Ti_To.upper_bound=0.198
model.solver='glpk'
sol = model.optimize()
sol.objective_value
fbasol = model.optimize()
b = show_map(fbasol,map_loc)
b

0.25475539999999997

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [84]:
model1=model.copy()
# Kncoking gene 'HGNC:8582' to simulate the PKU patients
model1.genes.get_by_id('HGNC:8582').knock_out()
model1.reactions.PHPYRte.upper_bound=0
model1.reactions.PHPYRte.lower_bound=0
sol = model1.optimize()
sol.objective_value
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

0.18734518083433543

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [85]:
model3=model1.copy()
model3.reactions.EX_tyr_L_e.lower_bound=-0.2
sol = model3.optimize()
sol.objective_value
fbasol = model3.optimize()
b = show_map(fbasol,map_loc)
b

0.18734518083433538

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [69]:
model2=model1.copy()
model2.reactions.EX_tyr_L_e.lower_bound=-0.2
model2.reactions.EX_phe_L_e.upper_bound=-0.04
model2.reactions.EX_phe_L_e.lower_bound=-0.04
sol = model2.optimize()
sol.objective_value
fbasol = model2.optimize()
b = show_map(fbasol,map_loc)
b

0.25367589999999995

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [68]:
model2.reactions.EX_phe_L_e.upper_bound=-0.04
model2.reactions.EX_phe_L_e.lower_bound=-0.04
model2.reactions.EX_tyr_L_e.lower_bound=-0.1
sol = model2.optimize()
sol.objective_value
fbasol = model2.optimize()
b = show_map(fbasol,map_loc)
b

0.25367589999999995

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

# Recon2.2plusthreeCompetition.json [Toxicity mechanism only]

In [66]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba
M = cobra.io.load_json_model('Recon2.2plusthreeCompetitive.json')
# deleting brain medium composition because this metabolite will uptake from blood
M.reactions.EX_co2_e_INB.lower_bound=0
M.reactions.EX_co2_e.lower_bound=0
M.reactions.EX_glc_e_INB.lower_bound=0
M.reactions.EX_h_e.lower_bound=0
M.reactions.EX_h_e_INB.lower_bound=0
M.reactions.EX_hco3_e_INB.lower_bound=0
M.reactions.EX_hco3_e.lower_bound=0
M.reactions.EX_his_L_e_INB.lower_bound=0
M.reactions.EX_ile_L_e_INB.lower_bound=0
M.reactions.EX_leu_L_e_INB.lower_bound=0
M.reactions.EX_lys_L_e_INB.lower_bound=0
M.reactions.EX_met_L_e_INB.lower_bound=0
M.reactions.EX_nh4_e_INB.lower_bound=0
M.reactions.EX_o2_e_INB.lower_bound=0
M.reactions.EX_phe_L_e_INB.lower_bound=0
M.reactions.EX_pi_e_INB.lower_bound=0
M.reactions.EX_so4_e_INB.lower_bound=0
M.reactions.EX_thr_L_e_INB.lower_bound=0
M.reactions.EX_trp_L_e_INB.lower_bound=0
M.reactions.EX_val_L_e_INB.lower_bound=0

In [67]:
from cobra.flux_analysis import pfba
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon2.2plusthreeCompetitivemap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

model=M.copy()

In [68]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = model1.problem.Objective(model.reactions.biomass_reaction_INB.flux_expression+
                                              model.reactions.get_by_id('3HLYTCL_INB').flux_expression+
                                              model.reactions.get_by_id('5HLTDL_INB').flux_expression,
    direction='max')
model.objective = quadratic_objective

In [69]:
# Changing the phe and tyr in the medium, because they also use for producing dopa.
model.reactions.EX_phe_L_e.lower_bound=-0.1
model.reactions.EX_phe_L_e.upper_bound=-0.1
model.reactions.EX_tyr_L_e.lower_bound=-0.02

In [71]:
model.reactions.EX_dopa_e_INB.upper_bound=1000
model.reactions.EX_adrnl_e_INB.upper_bound=1000
model.reactions.EX_srtn_e_INB.upper_bound=1000
## Other reactions to produce l-dopa is forbidded
model.reactions.TYRDOPO_INB.upper_bound=0
model.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model.reactions.biomass_reaction.lower_bound=0.05
model.reactions.biomass_reaction.upper_bound=0.05
model.reactions.biomass_reaction_INB.upper_bound=0.1
# minimal serotonin production
# model.reactions.get_by_id('5HLTDL_INB').lower_bound=0.098
### No PAH in the brain
model.reactions.PHETHPTOX2_INB.lower_bound=0
model.reactions.PHETHPTOX2_INB.upper_bound=0
model.reactions.r0399_INB.lower_bound=0
model.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
model.reactions.EX_TH_B_INB.upper_bound=1000
model.reactions.LdopaTR_TOBD.upper_bound=0 ##No levodopa from liver to blood.
# model.reactions.Ti_To.upper_bound=0.198
### Tyrosine hydroxylase competition
model.reactions.TH_AToB_INB.upper_bound=0.0572
model.solver='glpk'
sol = model.optimize()
sol.objective_value
fbasol = model.optimize()
b = show_map(fbasol,map_loc)
b

0.25513309999999995

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [72]:
model1=model.copy()
# Kncoking gene 'HGNC:8582' to simulate the PKU patients
model1.genes.get_by_id('HGNC:8582').knock_out()
model1.reactions.PHETA1.upper_bound=0
sol = model1.optimize()
fluxP=sol.fluxes.loc['PheTR_TOB']
print(fluxP)
if fluxP>=0.08:
    model1.reactions.Phe_TH_INB.lower_bound=0.055
sol.objective_value
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

0.08702650000000002


0.1735916577942005

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [44]:
model2=model1.copy()
model2.reactions.EX_tyr_L_e.lower_bound=-0.1
sol = model2.optimize()
sol.objective_value
fbasol = model2.optimize()
b = show_map(fbasol,map_loc)
b

0.2002041

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [47]:
model3=model2.copy()
model3.reactions.Phe_TH_INB.lower_bound=0
model3.reactions.EX_phe_L_e.upper_bound=-0.039
model3.reactions.EX_phe_L_e.lower_bound=-0.039
sol = model3.optimize()
sol.objective_value
fbasol = model3.optimize()
b = show_map(fbasol,map_loc)
b

0.25520410000000004

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

##Toxicity mechanism could describe the reduction of dopamine and could not describe the biomass synthesis (protein synthesis) in the brain when we add more tyrosine for PKU diseae. So, at least only toxicity mechanism could not add PKU disease.

# Recon2.2plusthreeCompetition.json [Toxicity mechanism and Competitive mechanism]

In [58]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba
M = cobra.io.load_json_model('Recon2.2plusthreeCompetitive.json')
# deleting brain medium composition because this metabolite will uptake from blood
M.reactions.EX_co2_e_INB.lower_bound=0
M.reactions.EX_co2_e.lower_bound=0
M.reactions.EX_glc_e_INB.lower_bound=0
M.reactions.EX_h_e.lower_bound=0
M.reactions.EX_h_e_INB.lower_bound=0
M.reactions.EX_hco3_e_INB.lower_bound=0
M.reactions.EX_hco3_e.lower_bound=0
M.reactions.EX_his_L_e_INB.lower_bound=0
M.reactions.EX_ile_L_e_INB.lower_bound=0
M.reactions.EX_leu_L_e_INB.lower_bound=0
M.reactions.EX_lys_L_e_INB.lower_bound=0
M.reactions.EX_met_L_e_INB.lower_bound=0
M.reactions.EX_nh4_e_INB.lower_bound=0
M.reactions.EX_o2_e_INB.lower_bound=0
M.reactions.EX_phe_L_e_INB.lower_bound=0
M.reactions.EX_pi_e_INB.lower_bound=0
M.reactions.EX_so4_e_INB.lower_bound=0
M.reactions.EX_thr_L_e_INB.lower_bound=0
M.reactions.EX_trp_L_e_INB.lower_bound=0
M.reactions.EX_val_L_e_INB.lower_bound=0

In [59]:
from cobra.flux_analysis import pfba
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon2.2plusthreeCompetitivemap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

model=M.copy()

In [60]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = model1.problem.Objective(model.reactions.biomass_reaction_INB.flux_expression+
                                              model.reactions.get_by_id('3HLYTCL_INB').flux_expression+
                                              model.reactions.get_by_id('5HLTDL_INB').flux_expression,
    direction='max')
model.objective = quadratic_objective

In [61]:
# Changing the phe and tyr in the medium, because they also use for producing dopa.
model.reactions.EX_phe_L_e.lower_bound=-0.1
model.reactions.EX_phe_L_e.upper_bound=-0.1
model.reactions.EX_tyr_L_e.lower_bound=-0.02

In [62]:
model.reactions.EX_dopa_e_INB.upper_bound=1000
model.reactions.EX_adrnl_e_INB.upper_bound=1000
model.reactions.EX_srtn_e_INB.upper_bound=1000
## Other reactions to produce l-dopa is forbidded
model.reactions.TYRDOPO_INB.upper_bound=0
model.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model.reactions.biomass_reaction.lower_bound=0.05
model.reactions.biomass_reaction.upper_bound=0.05
model.reactions.biomass_reaction_INB.upper_bound=0.1
# minimal serotonin production
# model.reactions.get_by_id('5HLTDL_INB').lower_bound=0.098
### No PAH in the brain
model.reactions.PHETHPTOX2_INB.lower_bound=0
model.reactions.PHETHPTOX2_INB.upper_bound=0
model.reactions.r0399_INB.lower_bound=0
model.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
model.reactions.EX_TH_B_INB.upper_bound=1000
model.reactions.LdopaTR_TOBD.upper_bound=0 ##No levodopa from liver to blood.
model.reactions.Ti_To.upper_bound=0.198
### Tyrosine hydroxylase competition
model.reactions.TH_AToB_INB.upper_bound=0.0572
model.solver='glpk'
sol = model.optimize()
sol.objective_value
fbasol = model.optimize()
b = show_map(fbasol,map_loc)
b

0.2547554000000001

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [63]:
model1=model.copy()
# Kncoking gene 'HGNC:8582' to simulate the PKU patients
model1.genes.get_by_id('HGNC:8582').knock_out()
model1.reactions.PHPYRbtBD.upper_bound=0
sol = model1.optimize()
fluxP=sol.fluxes.loc['PheTR_TOB']
print(fluxP)
if fluxP>=0.08:
    model1.reactions.Phe_TH_INB.lower_bound=0.055
sol.objective_value
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

0.08664880000000005


0.1735916577941996

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [64]:
model2=model1.copy()
model2.reactions.EX_tyr_L_e.lower_bound=-0.1
sol = model2.optimize()
sol.objective_value
fbasol = model2.optimize()
b = show_map(fbasol,map_loc)
b

0.19975540000000003

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

##Even we found that the biomass production increased in the brain, it sacrificed the serotonin production.

In [65]:
model3=model2.copy()
model3.reactions.Phe_TH_INB.lower_bound=0
model3.reactions.EX_phe_L_e.upper_bound=-0.039
model3.reactions.EX_phe_L_e.lower_bound=-0.039
sol = model3.optimize()
sol.objective_value
fbasol = model3.optimize()
b = show_map(fbasol,map_loc)
b

0.2547554000000001

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

##Finally, we found phe-deprivation conbining with tyrosine when it was needed is useful treatment.

# Recon2.2plus with exercise (Other treatment)

In [1]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba

In [2]:
from cobra.flux_analysis import pfba
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon2.2plusthreeCompetitivemap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Load Recon2.2Model with a simplified medium for the course
M = cobra.io.load_json_model('Recon2.2plusthreeCompetitive.json')

/Users/yanhualiu/Desktop/New_topic/model_building/model_building_exercise_by_yanhua/for different models/Competition between Phe, Tyr, Trp and L-dopa/Making brain specific model and complex model/utils/findBiomarkers.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [3]:
# deleting brain medium composition because this metabolite will uptake from blood
M.reactions.EX_co2_e_INB.lower_bound=0
M.reactions.EX_co2_e.lower_bound=0
M.reactions.EX_glc_e_INB.lower_bound=0
M.reactions.EX_h_e.lower_bound=0
M.reactions.EX_h_e_INB.lower_bound=0
M.reactions.EX_hco3_e_INB.lower_bound=0
M.reactions.EX_hco3_e.lower_bound=0
M.reactions.EX_his_L_e_INB.lower_bound=0
M.reactions.EX_ile_L_e_INB.lower_bound=0
M.reactions.EX_leu_L_e_INB.lower_bound=0
M.reactions.EX_lys_L_e_INB.lower_bound=0
M.reactions.EX_met_L_e_INB.lower_bound=0
M.reactions.EX_nh4_e_INB.lower_bound=0
M.reactions.EX_o2_e_INB.lower_bound=0
M.reactions.EX_phe_L_e_INB.lower_bound=0
M.reactions.EX_pi_e_INB.lower_bound=0
M.reactions.EX_so4_e_INB.lower_bound=0
M.reactions.EX_thr_L_e_INB.lower_bound=0
M.reactions.EX_trp_L_e_INB.lower_bound=0
M.reactions.EX_val_L_e_INB.lower_bound=0

In [4]:
model=M.copy()

In [5]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = model.problem.Objective(model.reactions.biomass_reaction_INB.flux_expression+
                                              model.reactions.get_by_id('3HLYTCL_INB').flux_expression+
                                              model.reactions.get_by_id('5HLTDL_INB').flux_expression,
    direction='max')
model.objective = quadratic_objective

In [6]:
# Changing the phe and tyr in the medium, because they also use for producing dopa.
model.reactions.EX_phe_L_e.lower_bound=-0.1
model.reactions.EX_phe_L_e.upper_bound=-0.1
model.reactions.EX_tyr_L_e.lower_bound=-0.1

In [7]:
model.reactions.EX_dopa_e_INB.upper_bound=1000
model.reactions.EX_adrnl_e_INB.upper_bound=1000
model.reactions.EX_srtn_e_INB.upper_bound=1000
## Other reactions to produce l-dopa is forbidded
model.reactions.TYRDOPO_INB.upper_bound=0
model.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model.reactions.biomass_reaction.lower_bound=0.05
model.reactions.biomass_reaction.upper_bound=0.05
## do not need too much biomass in the brain
model.reactions.biomass_reaction_INB.upper_bound=0.1
# No phenylpyruvate exchange in the brain
model.reactions.get_by_id('EX_phpyr_e_INB').upper_bound=0
# Serotonin production
model.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
### No PAH in the brain
model.reactions.PHETHPTOX2_INB.lower_bound=0
model.reactions.PHETHPTOX2_INB.upper_bound=0
model.reactions.r0399_INB.lower_bound=0
model.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
model.reactions.EX_TH_B_INB.upper_bound=1000
model.reactions.LdopaTR_TOBD.upper_bound=0 ##No levodopa from liver to blood.
model.reactions.Ti_To.upper_bound=0.198
model.solver='glpk'
sol = model.optimize()
sol.objective_value
fbasol = model.optimize()
b = show_map(fbasol,map_loc)
b

0.2547554000000001

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [8]:
model1=model.copy()
# Kncoking gene 'HGNC:8582' to simulate the PKU patients
model1.genes.get_by_id('HGNC:8582').knock_out()
model1.reactions.PHPYRte.upper_bound=0
model1.reactions.PHPYRte.lower_bound=0
model1.reactions.EX_phpyr_e_INB.upper_bound=0
sol = model1.optimize()
sol.objective_value

0.18734518083433543

In [9]:
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [10]:
#Remembering to add other amino acid
model1.reactions.EX_glc_e.lower_bound=-2##Do not need much glucsoe.
model1.reactions.EX_his_L_e.lower_bound=-0.2
model1.reactions.EX_ile_L_e.lower_bound=-0.2
model1.reactions.EX_leu_L_e.lower_bound=-0.2
model1.reactions.EX_lys_L_e.lower_bound=-0.2
model1.reactions.EX_met_L_e.lower_bound=-0.2
model1.reactions.EX_thr_L_e.lower_bound=-0.2
model1.reactions.EX_trp_L_e.lower_bound=-0.1
model1.reactions.EX_val_L_e.lower_bound=-0.2

In [11]:
model1.reactions.biomass_reaction.upper_bound=0.5
sol = model1.optimize()
sol.objective_value

0.24190296608223896

In [12]:
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

## More carriers for model

In [13]:
model2=model.copy()
# Kncoking gene 'HGNC:8582' to simulate the PKU patients
model2.genes.get_by_id('HGNC:8582').knock_out()
model2.reactions.PHPYRte.upper_bound=0
model2.reactions.PHPYRte.lower_bound=0
model2.reactions.EX_phpyr_e_INB.upper_bound=0
sol = model2.optimize()
sol.objective_value

0.18734518083433543

In [14]:
fbasol = model2.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [16]:
model2.reactions.Ti_To.upper_bound=0.3
sol = model2.optimize()
sol.objective_value
fbasol = model2.optimize()
b = show_map(fbasol,map_loc)
b

0.27405359999999995

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…